In [ ]:
import pickle
import os
import numpy as np
from scipy.signal import resample
import torch
import numpy as np
from torch.utils.data import Dataset, DataLoader
import torch.nn as nn
import torch.optim as optim
import torch.nn.functional as F
import math
from tqdm import tqdm
import matplotlib.pyplot as plt
from sklearn.metrics import f1_score, precision_score, recall_score, roc_auc_score
from scipy.interpolate import interp1d
import typing as tp
import random

# 假设 SEANetTransformerClassifier 和 create_mask 在导入的 seanet_style 模块中
# from seanet_style import SEANetTransformerClassifier, create_mask 
# 保持原有的导入，但需要确保这些类是可用的。

# --- Dataset Classes ---

class ClassificationDataset(Dataset):
    def __init__(self, input_files, output_files, min_window=1000, max_window=4000,
                 noise_stds=1e-6, transform_probability=0.95, step_size=50,
                 channel_wise_noise_stds=None,
                 scale_range=(0.9, 1.1),
                 num_warp_knots=4,
                 warp_scale=0.1):
        self.input_files = input_files
        self.output_files = output_files
        assert len(self.input_files) == len(self.output_files), "输入输出文件数量不匹配"

        self.file_metadata = []
        # 使用 np.load(f, mmap_mode='r') 节省内存
        self.input_memmaps = [np.load(f, mmap_mode='r') for f in self.input_files]
        self.output_memmaps = [np.load(f, mmap_mode='r') for f in self.output_files]

        self.min_window = min_window
        self.max_window = max_window
        self.step_size = step_size
        self.valid_window_sizes = list(range(min_window, max_window + 1, step_size))
        if not self.valid_window_sizes:
            self.valid_window_sizes = [min_window]

        self.noise_stds = noise_stds
        self.transform_prob = transform_probability
        self.channel_wise_noise_stds = channel_wise_noise_stds
        self.scale_range = scale_range

        self.num_warp_knots = num_warp_knots
        self.warp_scale = warp_scale

        self.total_windows = 0
        for i, (inp_mmap, out_mmap) in enumerate(zip(self.input_memmaps, self.output_memmaps)):
            channels, length = inp_mmap.shape

            num_windows_in_file = 0
            if length >= self.min_window:
                # 简单计算基于 min_window 和 step_size 的窗口数量
                num_windows_in_file = (length - self.min_window) // self.step_size + 1

            self.file_metadata.append({
                'input_shape': inp_mmap.shape,
                'output_shape': out_mmap.shape,
                'length': length,
                'channels': channels,
                'num_windows': num_windows_in_file,
                'start_idx_offset': self.total_windows
            })
            self.total_windows += num_windows_in_file

    def __len__(self):
        return self.total_windows

    def _time_warp(self, input_tensor, output_tensor):
        """
        Applies time warping to the input and output tensors.
        input_tensor: (C, L)
        output_tensor: (C', L)
        """
        length = input_tensor.shape[1]
        t = np.linspace(0, 1, length)

        knots_x = np.linspace(0, 1, self.num_warp_knots)
        knots_y = knots_x + np.random.normal(0, self.warp_scale, self.num_warp_knots)
        knots_y = np.sort(knots_y)

        knots_y[0] = 0
        knots_y[-1] = 1

        warp_func = interp1d(knots_x, knots_y, kind='cubic', fill_value="extrapolate")
        warped_t = warp_func(t)
        warped_t = np.clip(warped_t, 0, 1)

        warped_input_channels = []
        for i in range(input_tensor.shape[0]):
            interp_func = interp1d(t, input_tensor[i, :].numpy(), kind='linear', fill_value="extrapolate")
            warped_input_channels.append(interp_func(warped_t))
        warped_input = torch.from_numpy(np.array(warped_input_channels)).float()

        warped_output_channels = []
        for i in range(output_tensor.shape[0]):
            interp_func = interp1d(t, output_tensor[i, :].numpy(), kind='linear', fill_value="extrapolate")
            warped_output_channels.append(interp_func(warped_t))
        warped_output = torch.from_numpy(np.array(warped_output_channels)).float()

        # Reshape/pad to original length L
        if warped_input.shape[1] < length:
            pad_amount = length - warped_input.shape[1]
            warped_input = F.pad(warped_input, (0, pad_amount), 'constant', 0)
            warped_output = F.pad(warped_output, (0, pad_amount), 'constant', 0)
        elif warped_input.shape[1] > length:
            warped_input = warped_input[:, :length]
            warped_output = warped_output[:, :length]
        
        threshold = 0.5
        warped_output = (warped_output > threshold).float()
        return warped_input, warped_output


    def __getitem__(self, idx):
        current_offset = 0
        file_idx = -1
        local_idx = -1
        for i, meta in enumerate(self.file_metadata):
            if idx < current_offset + meta['num_windows']:
                file_idx = i
                local_idx = idx - current_offset
                break
            current_offset += meta['num_windows']

        if file_idx == -1:
            raise IndexError("Index out of bounds for dataset.")

        meta = self.file_metadata[file_idx]

        # 1. 随机选择窗口长度
        window_length = np.random.choice(self.valid_window_sizes)

        # 2. 窗口选择策略：基于步长对齐的基础起始点 + 随机偏移
        base_start = local_idx * self.step_size
        max_possible_start = max(0, meta['length'] - window_length)
        random_offset = np.random.randint(0, self.step_size)
        start = min(base_start + random_offset, max_possible_start)
        end = start + window_length

        if end > meta['length']:
            end = meta['length']
            start = max(0, end - window_length)

        input_data_np = self.input_memmaps[file_idx][:, start:end]
        output_data_np = self.output_memmaps[file_idx][:, start:end]

        input_window = torch.from_numpy(input_data_np).float()
        output_window = torch.from_numpy(output_data_np).float()

        if output_window.shape[0] > 1:
            output_window = output_window[0:1, :]

        # 3. 数据增强
        if np.random.rand() < self.transform_prob:
            # Random Scaling
            scale_factor = np.random.uniform(self.scale_range[0], self.scale_range[1])
            input_window = input_window * scale_factor

            # Randomly add noise
            if self.channel_wise_noise_stds is not None:
                # noise_scale is expected to be a tensor of shape (C, 1) or (C)
                noise_scale = self.channel_wise_noise_stds.to(input_window.device)
                if noise_scale.dim() == 1:
                    noise_scale = noise_scale.unsqueeze(-1) # (C, 1)
                
                # Expand noise_scale to match input_window.shape[1] for broadcasting
                noise = torch.randn_like(input_window) * noise_scale
            else:
                noise = torch.randn_like(input_window) * self.noise_stds

            input_window = input_window + noise

            input_window, output_window = self._time_warp(input_window, output_window)

        # 确保 output_window 长度与 input_window 匹配 (时间轴)
        if input_window.shape[-1] != output_window.shape[-1]:
             min_len = min(input_window.shape[-1], output_window.shape[-1])
             input_window = input_window[..., :min_len]
             output_window = output_window[..., :min_len]

        return input_window, output_window, input_window.shape[-1], output_window.shape[-1]


class ValidationDataset(Dataset):
    def __init__(self, input_files, output_files, chunk_size=1000):
        self.input_files = input_files
        self.output_files = output_files
        assert len(self.input_files) == len(self.output_files), "Input and output file counts mismatch."
        
        self.chunk_size = chunk_size
        self.data_chunks = []

        # 在 ValidationDataset 中，我们直接加载数据并切分成 chunk，不使用 mmap_mode='r'
        # 因为验证集通常较小，且需要立即切分
        for i, (input_file, output_file) in enumerate(zip(self.input_files, self.output_files)):
            inp_full_np = np.load(input_file)
            out_full_np = np.load(output_file)

            min_len = min(inp_full_np.shape[1], out_full_np.shape[1])
            input_full_np = inp_full_np[:, :min_len]
            output_full_np = out_full_np[:, :min_len]

            if output_full_np.shape[0] > 1:
                output_full_np = output_full_np[0:1, :]

            total_input_len = input_full_np.shape[1]
            
            for start_idx in range(0, total_input_len, chunk_size):
                end_idx = min(start_idx + chunk_size, total_input_len)
                
                input_chunk = torch.from_numpy(input_full_np[:, start_idx:end_idx]).float()
                output_chunk = torch.from_numpy(output_full_np[:, start_idx:end_idx]).float()
                
                self.data_chunks.append((input_chunk, output_chunk, input_chunk.shape[1], output_chunk.shape[1]))

    def __len__(self):
        return len(self.data_chunks)

    def __getitem__(self, idx):
        return self.data_chunks[idx]

# --- Helper Functions (collate_fn and metrics functions remain the same) ---

def collate_fn(batch):
    inputs, outputs, input_lengths_orig, output_lengths_orig = zip(*batch)
    
    input_lengths = list(input_lengths_orig)
    output_lengths = list(output_lengths_orig)
    
    max_input_len = max(input_lengths)
    padded_inputs = torch.stack([
        torch.nn.functional.pad(x, (0, max_input_len - x.shape[-1])) 
        for x in inputs
    ])
    
    max_output_len = max(output_lengths)
    padded_outputs = torch.stack([
        torch.nn.functional.pad(y, (0, max_output_len - y.shape[-1]))
        for y in outputs
    ])
    
    return padded_inputs, padded_outputs, input_lengths, output_lengths

def calculate_classification_metrics_optimized(tp, fp, tn, fn, all_predictions_probs_flat, all_targets_flat):
    """
    Calculates classification metrics using accumulated confusion matrix components.
    Includes ROC AUC calculation which still requires all probabilities and targets.
    """
    # Calculate F1, Precision, Recall from TP, FP, TN, FN
    precision = tp / (tp + fp + 1e-8)
    recall = tp / (tp + fn + 1e-8)
    f1 = 2 * (precision * recall) / (precision + recall + 1e-8)
    
    # ROC AUC calculation
    if len(np.unique(all_targets_flat)) > 1 and len(all_predictions_probs_flat) > 0:
        roc_auc = roc_auc_score(all_targets_flat, all_predictions_probs_flat)
    else:
        roc_auc = float('nan') 

    return f1, precision, recall, roc_auc

def evaluate_model(model, dataloader, criterion, num_output_channels, device, threshold=0.5, max_auc_samples=500000):
    model.eval()
    total_loss = 0.0
    total_valid_elements = 0
    
    true_positives = 0
    false_positives = 0
    true_negatives = 0
    false_negatives = 0

    all_valid_predictions_probs_for_auc = []
    all_valid_targets_for_auc = []

    with torch.no_grad():
        progress_bar_val = tqdm(dataloader, desc="Validation")
        for batch in progress_bar_val:
            inputs, targets, input_lengths, output_lengths = batch
            inputs, targets = inputs.to(device), targets.to(device)

            targets = targets.float()

            predictions_logits = model(inputs, original_lengths=input_lengths)
            
            max_output_len = predictions_logits.size(-1)
            output_mask_batch_time = create_mask(
                [min(l, max_output_len) for l in output_lengths], 
                max_output_len,
                device
            )

            expanded_output_mask = output_mask_batch_time.unsqueeze(1) 

            predictions_logits_flat = predictions_logits.flatten()
            targets_flat = targets.flatten()

            valid_elements_mask_flat = (~expanded_output_mask).flatten()

            valid_predictions_logits = predictions_logits_flat[valid_elements_mask_flat]
            valid_targets = targets_flat[valid_elements_mask_flat]

            if valid_targets.numel() > 0:
                loss_tensor = criterion(valid_predictions_logits, valid_targets)
                loss = loss_tensor.mean()
                total_loss += loss.item() * valid_targets.numel()
                total_valid_elements += valid_targets.numel()

                valid_predictions_probs = torch.sigmoid(valid_predictions_logits)
                valid_predictions_binary = (valid_predictions_probs >= threshold).int()

                true_positives += ((valid_predictions_binary == 1) & (valid_targets == 1)).sum().item()
                false_positives += ((valid_predictions_binary == 1) & (valid_targets == 0)).sum().item()
                true_negatives += ((valid_predictions_binary == 0) & (valid_targets == 0)).sum().item()
                false_negatives += ((valid_predictions_binary == 0) & (valid_targets == 1)).sum().item()

                # --- 采样部分 START ---
                current_batch_size_valid = valid_targets.numel()
                if len(all_valid_predictions_probs_for_auc) < max_auc_samples:
                    # 如果总样本数还没达到上限，直接添加
                    all_valid_predictions_probs_for_auc.extend(valid_predictions_probs.cpu().tolist())
                    all_valid_targets_for_auc.extend(valid_targets.cpu().tolist())
                elif current_batch_size_valid > 0:
                    # 如果已达到上限，从当前批次中随机采样一部分
                    num_to_sample = min(current_batch_size_valid, max_auc_samples // 10)
                    if num_to_sample > 0:
                        indices = torch.randperm(current_batch_size_valid)[:num_to_sample]
                        all_valid_predictions_probs_for_auc.extend(valid_predictions_probs[indices].cpu().tolist())
                        all_valid_targets_for_auc.extend(valid_targets[indices].cpu().tolist())
                        
                        if len(all_valid_predictions_probs_for_auc) > max_auc_samples:
                            all_valid_predictions_probs_for_auc = all_valid_predictions_probs_for_auc[-max_auc_samples:]
                            all_valid_targets_for_auc = all_valid_targets_for_auc[-max_auc_samples:]
                # --- 采样部分 END ---

            else:
                loss = torch.tensor(0.0).to(device)

    avg_loss = total_loss / (total_valid_elements + 1e-8) if total_valid_elements > 0 else float('nan')
    
    sampled_probs_np = np.array(all_valid_predictions_probs_for_auc)
    sampled_targets_np = np.array(all_valid_targets_for_auc)

    f1, precision, recall, roc_auc = calculate_classification_metrics_optimized(
        true_positives, false_positives, true_negatives, false_negatives,
        sampled_probs_np, sampled_targets_np
    )
    
    return avg_loss, f1, precision, recall, roc_auc

def plot_metrics(train_losses, val_losses, train_f1_scores, val_f1_scores,
                 train_precisions, val_precisions, train_recalls, val_recalls,
                 train_roc_aucs, val_roc_aucs,
                 num_epochs, save_dir="."):
    
    epochs = range(1, num_epochs + 1)

    fig, axes = plt.subplots(2, 3, figsize=(20, 12))
    fig.suptitle('Training and Validation Metrics per Epoch', fontsize=16)

    axes[0, 0].plot(epochs, train_losses, label='Training Loss')
    axes[0, 0].plot(epochs, val_losses, label='Validation Loss')
    axes[0, 0].set_title('Binary Cross Entropy Loss')
    axes[0, 0].set_xlabel('Epoch')
    axes[0, 0].set_ylabel('Loss')
    axes[0, 0].legend()
    axes[0, 0].grid(True)

    axes[0, 1].plot(epochs, train_f1_scores, label='Training F1-score', color='orange')
    axes[0, 1].plot(epochs, val_f1_scores, label='Validation F1-score', color='red')
    axes[0, 1].set_title('F1-score')
    axes[0, 1].set_xlabel('Epoch')
    axes[0, 1].set_ylabel('F1-score')
    axes[0, 1].legend()
    axes[0, 1].grid(True)

    axes[0, 2].plot(epochs, train_precisions, label='Training Precision', color='green', linestyle='--')
    axes[0, 2].plot(epochs, val_precisions, label='Validation Precision', color='green')
    axes[0, 2].set_title('Precision')
    axes[0, 2].set_xlabel('Epoch')
    axes[0, 2].set_ylabel('Precision')
    axes[0, 2].legend()
    axes[0, 2].grid(True)

    axes[1, 0].plot(epochs, train_recalls, label='Training Recall', color='purple', linestyle='--')
    axes[1, 0].plot(epochs, val_recalls, label='Validation Recall', color='purple')
    axes[1, 0].set_title('Recall')
    axes[1, 0].set_xlabel('Epoch')
    axes[1, 0].set_ylabel('Recall')
    axes[1, 0].legend()
    axes[1, 0].grid(True)

    axes[1, 1].plot(epochs, train_roc_aucs, label='Training ROC AUC', color='brown', linestyle='--')
    axes[1, 1].plot(epochs, val_roc_aucs, label='Validation ROC AUC', color='brown')
    axes[1, 1].set_title('ROC AUC')
    axes[1, 1].set_xlabel('Epoch')
    axes[1, 1].set_ylabel('ROC AUC Score')
    axes[1, 1].legend()
    axes[1, 1].grid(True)
    
    axes[1, 2].axis('off')

    plt.tight_layout(rect=[0, 0.03, 1, 0.95])
    plt.savefig(os.path.join(save_dir, 'classification_metrics_combined.png'))
    plt.close(fig)


/home/lta/miniconda3/envs/libribrain_2025/lib/python3.13/site-packages/torch/nn/modules/transformer.py:382: UserWarning: enable_nested_tensor is True, but self.use_nested_tensor is False because encoder_layer.self_attn.batch_first was not True(use batch_first for better inference performance)
  warnings.warn(
Epoch 1/100 (Train): 100%|██████████████████████████████| 2307/2307 [15:19<00:00,  2.51it/s, avg_loss=0.289, loss=0.272]



--- Epoch 1/100 Training Summary ---
Train Average Loss (BCE): 0.288559
Train F1-score: 0.776877
Train Precision: 0.875939
Train Recall: 0.697945
Train ROC AUC: 0.500652


Validation: 100%|█████████████████████████████████████████████████████████████████████| 139/139 [00:06<00:00, 21.36it/s]



--- Epoch 1/100 Validation Summary ---
Validation BCE Loss: 0.269950
Validation F1-score: 0.819991
Validation Precision: 0.886354
Validation Recall: 0.762873
Validation ROC AUC: 0.813890
Current Learning Rate: 0.000100
New best model saved with validation ROC AUC: 0.813890


Epoch 2/100 (Train): 100%|███████████████████████████████| 2307/2307 [15:30<00:00,  2.48it/s, avg_loss=0.256, loss=0.22]


...

Epoch 83/100 (Train): 100%|█████████████████████████████| 2307/2307 [15:43<00:00,  2.45it/s, avg_loss=0.166, loss=0.172]



--- Epoch 83/100 Training Summary ---
Train Average Loss (BCE): 0.165980
Train F1-score: 0.899371
Train Precision: 0.946021
Train Recall: 0.857105
Train ROC AUC: 0.931551


Validation: 100%|█████████████████████████████████████████████████████████████████████| 139/139 [00:04<00:00, 28.01it/s]



--- Epoch 83/100 Validation Summary ---
Validation BCE Loss: 0.213004
Validation F1-score: 0.872517
Validation Precision: 0.918132
Validation Recall: 0.831220
Validation ROC AUC: 0.887836
Current Learning Rate: 0.000050


Epoch 84/100 (Train):   7%|██▏                           | 167/2307 [01:11<13:04,  2.73it/s, avg_loss=0.164, loss=0.141]

In [ ]:
# --- Dynamic STD Calculation and Data Splitting ---

# 统一的数据根目录
MEG_ROOT = "./libribrain/data/meg_data_npy"
LABEL_ROOT = "./libribrain/data/labels_npy"
VALIDATION_SPLIT = 0.2
NUM_CHANNELS = 48 # 固定通道数

# 获取所有文件对
input_files = sorted([os.path.join(MEG_ROOT, f) for f in os.listdir(MEG_ROOT) if f.endswith('.npy')])
output_files = sorted([os.path.join(LABEL_ROOT, f) for f in os.listdir(LABEL_ROOT) if f.endswith('.npy')])

# 确保文件数量匹配
assert len(input_files) == len(output_files), "输入和输出文件夹中的文件数量不匹配"



In [ ]:
# --- Step 1: 动态计算所有 MEG 文件的通道标准差 (STD) ---
print("--- Calculating Channel Standard Deviations ---")
channel_sum = np.zeros(NUM_CHANNELS, dtype=np.float64)
channel_sq_sum = np.zeros(NUM_CHANNELS, dtype=np.float64)
channel_count = np.zeros(NUM_CHANNELS, dtype=np.int64)

for input_file in tqdm(input_files, desc="Processing files for STD calculation"):
    # 使用 mmap_mode='r' 减少内存占用
    data = np.load(input_file, mmap_mode='r').astype(np.float64)
    
    # 假设 MEG 数据是 (Channels, Time) 形状，即 (48, T)
    if data.shape[0] != NUM_CHANNELS:
        # 如果是 (T, 48)，则转置为 (48, T)
        if data.shape[1] == NUM_CHANNELS:
            data = data.T
        else:
            print(f"Warning: File {input_file} has unexpected shape {data.shape}. Skipping.")
            continue
            
    # data 形状现在是 (48, T)
    T = data.shape[1]
    
    channel_sum += np.sum(data, axis=1) # 沿着时间轴求和
    channel_sq_sum += np.sum(np.square(data), axis=1)
    channel_count += T # 每个通道增加了 T 个样本

# 避免除以零
safe_channel_count = np.where(channel_count > 0, channel_count, 1)

# 计算 mean 和 std
mean = channel_sum / safe_channel_count
# 使用 E[X^2] - (E[X])^2 公式计算方差
variance = (channel_sq_sum / safe_channel_count) - (mean**2)
# 确保方差非负
std = np.sqrt(np.maximum(variance, 0))

# 转换为 torch tensor，并调整形状为 (C, 1) 以便在 Dataset 中广播
channel_stds = torch.from_numpy(std).float().unsqueeze(1)
print("--- STD Calculation Complete ---")
print(f"Calculated Channel STDs shape: {channel_stds.shape}")
print(f"Example STD (Channel 0): {channel_stds[0].item():.6f}")


In [ ]:
# --- Step 2: 划分训练集和验证集文件列表 ---

# 将输入文件和输出文件打包成元组列表
file_pairs = list(zip(input_files, output_files))
random.seed(42) # 保证每次划分结果一致
random.shuffle(file_pairs)

num_total = len(file_pairs)
num_val = int(num_total * VALIDATION_SPLIT)
num_train = num_total - num_val

train_file_pairs = file_pairs[num_val:]
val_file_pairs = file_pairs[:num_val]

train_input_files, train_output_files = zip(*train_file_pairs)
val_input_files, val_output_files = zip(*val_file_pairs)

print(f"Total files: {num_total}, Train files: {num_train}, Validation files: {num_val}")


In [ ]:
# --- Step 3: 创建 Dataset 和 Dataloader ---

# Create training dataset and dataloader
dataset = ClassificationDataset(
    list(train_input_files), 
    list(train_output_files), 
    min_window=1000, 
    max_window=5000, 
    step_size=1200, 
    channel_wise_noise_stds=(channel_stds * 0.1) # 使用计算得到的 STD
)
dataloader = DataLoader(dataset, collate_fn=collate_fn, batch_size=16, shuffle=True, num_workers=4, pin_memory=True) # 增加 num_workers

# Create validation dataset and dataloader
validation_dataset = ValidationDataset(list(val_input_files), list(val_output_files))
validation_dataloader = DataLoader(validation_dataset, collate_fn=collate_fn, batch_size=16, shuffle=False)


In [ ]:
# --- Model and Training Setup ---

num_epochs = 100
best_roc_auc = -1.0
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')

# Model parameters
model_params = {
    'input_channels': 48,
    'sampling_rate': 250,
    'encoder_dimension': 16,
    'encoder_n_filters': 8,
    'encoder_ratios': [5, 5, 2],
    'transformer_n_heads': 4,
    'transformer_n_layers': 1,
    'transformer_dim_feedforward': 64,
    'transformer_dropout': 0.1,
    'decoder_out_channels': 1,
    'activation': 'GELU',
    'activation_params': {},
    'norm': 'InstanceNorm1d',
    'norm_params': {},
    'n_residual_layers': 1,
    'kernel_size': 7,
    'last_kernel_size': 7,
    'residual_kernel_size': 3,
    'dilation_base': 2,
    'compress': 2,
    'true_skip': False
}
model_path = "./libribrain/speech_detection.pth"
# **注意：你需要确保 SEANetTransformerClassifier 和 create_mask 已经被正确定义或导入**
# 由于您的代码片段中没有它们，我假设它们是可用的。
# 实际运行时请确保它们在 `from seanet_style import SEANetTransformerClassifier, create_mask` 中。
model = SEANetTransformerClassifier(**model_params).to(device).float()

# Using BCEWithLogitsLoss for binary classification
# Assuming the positive class (label 1) proportion is 0.7464 (1 - 0.2536), or vice versa.
# The original code used 0.2536 / (1 - 0.2536), suggesting 0.2536 is the proportion of negative samples (or the target distribution is heavily skewed).
# Let's stick to the ratio in the original code, assuming 0.2536 is P(Negative)
# P(Positive) = 1 - P(Negative) = 1 - 0.2536 = 0.7464
# pos_weight = P(Negative) / P(Positive) = 0.2536 / 0.7464 ≈ 0.3398
pos_weight_value = 0.2536 / (1 - 0.2536) # Value from original code (approx 0.3398)
pos_weight = torch.tensor([pos_weight_value]).to(device)

criterion = nn.BCEWithLogitsLoss(pos_weight=pos_weight, reduction='none')
optimizer = optim.AdamW(model.parameters(), lr=1e-4, weight_decay=1e-5)
scheduler = optim.lr_scheduler.ReduceLROnPlateau(optimizer, mode='max', patience=10, factor=0.5, min_lr=1e-8)
num_channels = model_params['decoder_out_channels']

train_losses = []
train_f1_scores = []
train_precisions = []
train_recalls = []
train_roc_aucs = []

val_losses = []
val_f1_scores = []
val_precisions = []
val_recalls = []
val_roc_aucs = []

classification_threshold = 0.5
max_auc_samples = 50000 # 减少训练集 AUC 采样以避免内存问题


In [ ]:
# --- Training Loop ---

for epoch in range(num_epochs):
    # --- Training Phase ---
    model.train()
    running_loss_sum = 0.0
    total_train_valid_elements = 0
    
    train_true_positives = 0
    train_false_positives = 0
    train_true_negatives = 0
    train_false_negatives = 0

    all_train_valid_predictions_probs_for_auc = []
    all_train_valid_targets_for_auc = []
    
    progress_bar = tqdm(dataloader, desc=f"Epoch {epoch+1}/{num_epochs} (Train)")
    
    for batch in progress_bar:
        inputs, targets, input_lengths, output_lengths = batch
        inputs, targets = inputs.to(device), targets.to(device)
        targets = targets.float()
        
        optimizer.zero_grad()
        predictions_logits = model(inputs, original_lengths=input_lengths)
        
        max_output_len = predictions_logits.size(-1)
        output_mask_batch_time = create_mask(
            [min(l, max_output_len) for l in output_lengths], 
            max_output_len,
            device
        )

        expanded_output_mask = output_mask_batch_time.unsqueeze(1) 

        predictions_logits_flat = predictions_logits.flatten()
        targets_flat = targets.flatten()

        valid_elements_mask_flat = (~expanded_output_mask).flatten()

        valid_predictions_logits = predictions_logits_flat[valid_elements_mask_flat]
        valid_targets = targets_flat[valid_elements_mask_flat]

        if valid_targets.numel() > 0:
            loss = criterion(valid_predictions_logits, valid_targets).mean()
            loss.backward()
            torch.nn.utils.clip_grad_norm_(model.parameters(), 1.0)
            optimizer.step()
            
            running_loss_sum += loss.item() * valid_targets.numel()
            total_train_valid_elements += valid_targets.numel()
            
            with torch.no_grad():
                valid_predictions_probs = torch.sigmoid(valid_predictions_logits)
                valid_predictions_binary = (valid_predictions_probs >= classification_threshold).int()

                train_true_positives += ((valid_predictions_binary == 1) & (valid_targets == 1)).sum().item()
                train_false_positives += ((valid_predictions_binary == 1) & (valid_targets == 0)).sum().item()
                train_true_negatives += ((valid_predictions_binary == 0) & (valid_targets == 0)).sum().item()
                train_false_negatives += ((valid_predictions_binary == 0) & (valid_targets == 1)).sum().item()

                # For ROC AUC in training: sample/truncate
                if len(all_train_valid_predictions_probs_for_auc) < max_auc_samples:
                    all_train_valid_predictions_probs_for_auc.extend(valid_predictions_probs.cpu().tolist())
                    all_train_valid_targets_for_auc.extend(valid_targets.cpu().tolist())
                # Note: For training, simpler truncation/no-more-sampling approach is often used 
                # because the full train AUC is too slow/memory intensive.
        else:
            loss = torch.tensor(0.0).to(device)
            
        avg_loss_display = running_loss_sum / (total_train_valid_elements + 1e-8)
        progress_bar.set_postfix(
            loss=loss.item(),
            avg_loss=avg_loss_display
        )
            
    epoch_avg_loss = running_loss_sum / (total_train_valid_elements + 1e-8) if total_train_valid_elements > 0 else float('nan')

    train_f1, train_precision, train_recall, train_roc_auc = calculate_classification_metrics_optimized(
        train_true_positives, train_false_positives, train_true_negatives, train_false_negatives,
        np.array(all_train_valid_predictions_probs_for_auc),
        np.array(all_train_valid_targets_for_auc)
    )
    
    train_losses.append(epoch_avg_loss)
    train_f1_scores.append(train_f1)
    train_precisions.append(train_precision)
    train_recalls.append(train_recall)
    train_roc_aucs.append(train_roc_auc)

    print(f"\n--- Epoch {epoch+1}/{num_epochs} Training Summary ---")
    print(f"Train Average Loss (BCE): {epoch_avg_loss:.6f}")
    print(f"Train F1-score: {train_f1:.6f}")
    print(f"Train Precision: {train_precision:.6f}")
    print(f"Train Recall: {train_recall:.6f}")
    print(f"Train ROC AUC (Sampled): {train_roc_auc:.6f}")

    # --- Validation Phase ---
    # Max AUC samples for validation is larger as per original code
    val_loss, val_f1, val_precision, val_recall, val_roc_auc = evaluate_model(
        model, validation_dataloader, criterion, num_channels, device, classification_threshold, max_auc_samples=500000)
    
    val_losses.append(val_loss)
    val_f1_scores.append(val_f1)
    val_precisions.append(val_precision)
    val_recalls.append(val_recall)
    val_roc_aucs.append(val_roc_auc)


    print(f"\n--- Epoch {epoch+1}/{num_epochs} Validation Summary ---")
    print(f"Validation BCE Loss: {val_loss:.6f}")
    print(f"Validation F1-score: {val_f1:.6f}")
    print(f"Validation Precision: {val_precision:.6f}")
    print(f"Validation Recall: {val_recall:.6f}")
    print(f"Validation ROC AUC: {val_roc_auc:.6f}")
    print(f"Current Learning Rate: {optimizer.param_groups[0]['lr']:.6f}")
    
    scheduler.step(val_roc_auc)
    if val_roc_auc > best_roc_auc:
        best_roc_auc = val_roc_auc
        os.makedirs("./libribrain", exist_ok=True)
        torch.save(model.state_dict(), f"./libribrain/speech_detection.pth")
        print(f"New best model saved with validation ROC AUC: {best_roc_auc:.6f}")
        
print("\nTraining complete!")
torch.save(model.state_dict(), f"./libribrain/speech_detection_final.pth")
os.makedirs("./libribrain", exist_ok=True)
plot_metrics(train_losses, val_losses, train_f1_scores, val_f1_scores,
             train_precisions, val_precisions, train_recalls, val_recalls,
             train_roc_aucs, val_roc_aucs,
             num_epochs, save_dir="./libribrain/")
print(f"Combined classification metrics plot saved to ./libribrain/classification_metrics_combined.png")